In [113]:
import pandas as pd
import torch
import transformers
from transformers import AutoTokenizer
import datasets
from typing import Any, Tuple


In [114]:
path: str = "/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/processed/wn_neg_processed/debug.txt"

In [115]:
model_str: str = 'prajjwal1/bert-small'

In [ ]:
tok = AutoTokenizer.from_pretrained('roberta-base')

In [116]:
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(
            model_str,
            use_fast=True,
            max_len=512)

In [117]:
tokenizer.vocab_size

30522

In [118]:
tokenizer.add_tokens(["[REF-BEG]", "[REF-END]"])

2

In [ ]:
def tokenize_dataset(elem: dict) -> dict:
    """Tokenize Dataset.

    Atomic function applied to each instance of the dataset.

    :param elem: Element of the dataset.
    :returns: Dictionary including 'input_ids', 'attention_mask' and
        'labels'.
    """
    attention_mask: list = []
    input_ids: list = []
    labels: list = []
    for elem_masked, elem_unmasked in zip(elem['x'], elem['y']):
        masked: torch.Tensor = tokenizer(elem_masked, return_tensors='pt')
        unmasked: torch.Tensor = tokenizer(elem_unmasked, return_tensors='pt')
        unm: torch.Tensor = unmasked['input_ids']
        msk: torch.Tensor = masked['input_ids']
        att: torch.Tensor = masked['attention_mask']
        if masked['input_ids'].shape != unmasked['input_ids'].shape:
            msk, att = equalize_data(unm, msk, tokenizer.mask_token_id)
        unm[(msk == unm)] = -100
        attention_mask.append(att.squeeze())
        input_ids.append(msk.squeeze().long().tolist())
        labels.append(unm.squeeze().long().tolist())
    result: dict = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }
    return result

In [ ]:
def equalize_data(
        unmasked: torch.Tensor,
        masked: torch.Tensor,
        mask_token_id: int) -> Tuple[torch.Tensor, torch.Tensor]:
    """Add Mask Token to Shorter List.

    Sometimes it happens, that a masked word consists of two tokens (e.g.
    'foul ball' -> '<mask>'). To have equal length for masking, it is necessary,
    to have the same length. Thus, more masked tokens are added to to the
    masked sequence to account for the split (e.g. '<mask>' -> '<mask> <mask>').
    This is done in index space ([..., 50296, ...]-> [..., 50296, 50296, ...]).

    :param unmasked: Unmasked sequence (the longer sequence).
    :param masked: Masked sequence (the shorter sequence.)
    :param mask_token_id: Id of masked token from tokenizer.
    :returns: Equalized dataset.
    """
    ind: int = (masked == tokenizer.masked_token_id).nonzero(as_tuple=True)[1].item()
    diff: int = unmasked.shape[1] - masked.shape[1] + 1
    return (
        torch.cat(
            (
                masked[0][0:ind],
                torch.tensor([mask_token_id] * diff),
                masked[0][(ind + 1)::]
            ), 0).unsqueeze(0),
            torch.Tensor([1] * unmasked.shape[1]).to(torch.int8))

In [119]:
tokenizer.mask_token_id

103

In [120]:
len(tokenizer.)

SyntaxError: invalid syntax (3934011322.py, line 1)

In [121]:
ds = pd.read_csv(path, header=None)
ds.columns = ['text']
ds

,text
0,This is a natural object and not an artifact[M...
1,This is a natural object[MASK] and not an arti...
2,This is an artifact and not a natural object[M...
3,This is an artifact[MASK] and not a natural ob...
4,This is an overachievement and not an underach...
...,...
78,This is a deceleration and not an acceleration...
79,This is a deceleration[MASK] and not an accele...
80,This is an opening and not a closing[MASK].[RE...
81,This is an opening[MASK] and not a closing.[RE...


In [ ]:
end_index: int = 30523
begin_index: int = 30522

tokenized = tokenizer(ds.iloc[74].text, return_tensors='pt')
tokenized = {k : v.squeeze() for k, v in tokenized.items()}

if (end_index in tokenized['input_ids']) and (begin_index in tokenized['input_ids']):
    bool_vector = torch.ones(len(tokenized['input_ids']), dtype=torch.bool)
    begin, end = ((tokenized['input_ids'].squeeze() == begin_index).nonzero(as_tuple=True)[0])[0].item(), ((tokenized['input_ids'].squeeze() == end_index).nonzero(as_tuple=True)[0])[0].item()
    rm = torch.zeros(end + 1 - begin, dtype=torch.bool)
    bool_vector[torch.Tensor(range(begin, end + 1)).long()] = rm
    orig = torch.masked_select(tokenized['input_ids'], bool_vector)
    label = torch.masked_select(tokenized['input_ids'], ~bool_vector)
    label = label[1::]
    label = label[:-1]
    ind: int = (orig == tokenizer.mask_token_id).nonzero(as_tuple=True)[0].item()
    label_f = (torch.zeros(ind) - 100, label, torch.zeros(len(orig[(ind + 1):: ])) - 100)
    elem = (orig[0:ind], torch.tensor([tokenizer.mask_token_id] * len(label)), orig[(ind + 1):: ])
    elem, label_f = torch.concat(elem).unsqueeze(0), torch.concat(label_f).unsqueeze(0)
    attention_mask: torch.Tensor = torch.ones(elem.shape[1]).unsqueeze(0)
elem, label_f, attention_mask

(tensor([[ 101, 2023, 2003, 2019, 2248, 3462, 1998, 2025, 1037, 4968, 3462,  103,
           103, 1012,  102]]),
 tensor([[-100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
          -100., 4968., 3462., -100., -100.]]),
 tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [ ]:
tokenized

{'input_ids': tensor([  101,  2023,  2003,  2019,  2248,  3462,  1998,  2025,  1037,  4968,
          3462,   103,  1012, 30522,  4968,  3462, 30523,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [ ]:
def mask(inputs):
    labels = inputs.clone()
    # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
    probability_matrix = torch.full(labels.shape, 0.15)
    print(labels.tolist())
    special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
    special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

In [ ]:
mask(tokenizer('amelia was a bitch , i knew that quite well .', return_tensors='pt')['input_ids'])

[[101, 11556, 2001, 1037, 7743, 1010, 1045, 2354, 2008, 3243, 2092, 1012, 102]]


(tensor([[  101, 11556,  2001,  1037, 17824,   103,  1045,  2354,  2008,  3243,
           2092,  1012,   102]]),
 tensor([[-100, -100, -100, -100, 7743, 1010, -100, -100, -100, -100, -100, -100,
          -100]]))

# Final Function

In [178]:
def process_dataset(inputs: dict, end_index: int = 30523, begin_index: int = 30522, mlm_probabilty: float =0.15):
    """"""
    tokenized: dict = tokenizer(inputs['text'], return_tensors='pt')
    if (end_index in tokenized['input_ids']) and (begin_index in tokenized['input_ids']):
        tokenized = {k : v.squeeze() for k, v in tokenized.items()}

        # Find masked word in reference sequence
        bool_vector: torch.Tensor = torch.ones(len(tokenized['input_ids']), dtype=torch.bool)
        begin: int = ((tokenized['input_ids'].squeeze() == begin_index).nonzero(as_tuple=True)[0])[0].item()
        end: int = ((tokenized['input_ids'].squeeze() == end_index).nonzero(as_tuple=True)[0])[0].item()
        rm: torch.Tensor = torch.zeros(end + 1 - begin, dtype=torch.bool)
        bool_vector[torch.Tensor(range(begin, end + 1)).long()] = rm

        # Get sentence (masked) and label (known word)
        orig: torch.Tensor = torch.masked_select(tokenized['input_ids'], bool_vector)
        label: torch.Tensor = torch.masked_select(tokenized['input_ids'], ~bool_vector)
        # Remove special tokens around label
        label = label[1::]
        label = label[:-1]

        # Get split sequences 
        ind: int = (orig == tokenizer.mask_token_id).nonzero(as_tuple=True)[0].item()
        label_f: tuple = (torch.zeros(ind, dtype=torch.long) - 100, label, torch.zeros(len(orig[(ind + 1):: ]), dtype=torch.long) - 100)
        elem: tuple = (orig[0:ind], torch.tensor([tokenizer.mask_token_id] * len(label)), orig[(ind + 1):: ])

        # Concatenate splitted sequences and prepare for return
        elem: torch.Tensor = torch.concat(elem)
        label_f: torch.Tensor = torch.concat(label_f)
        attention_mask: torch.Tensor = torch.ones(elem.shape, dtype=torch.long)

        # Apply padding
        elem = torch.cat((elem, torch.Tensor([tokenizer.pad_token_type_id] * (512 - len(elem))).long()))
        label_f = torch.cat((label_f, (torch.zeros(512 - len(label_f), dtype=torch.long) - 100)))
        attention_mask = torch.cat((attention_mask, torch.zeros(512 - len(attention_mask), dtype=torch.long)))
        return_dict: dict = {
            'input_ids': elem,
            'labels': label_f,
            'attention_mask': attention_mask}
        return return_dict
    else:
        # MLM-Masking, according to Devlin et al. 2018.
        input_ids = tokenized['input_ids']
        labels = input_ids.clone()
        # Sample mlm_probability% of all appropriate tokens.
        probability_matrix = torch.full(labels.shape, mlm_probabilty)
        special_tokens_mask = [
                tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
            ]
        special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)

        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100

        # 80 % will be masked
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        input_ids[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

        # 15 % will be randomly filled.
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
        input_ids[indices_random] = random_words[indices_random]

        return_dict: dict = {
            'input_ids': torch.cat((input_ids, tokenizer.pad_token_type_id*  torch.ones((512 - input_ids.shape[1]), dtype=torch.long).unsqueeze(0)), 1).squeeze(),
            'labels': torch.cat((labels, torch.zeros(512 - labels.shape[1], dtype=torch.long).unsqueeze(0)),1 ).squeeze(),
            'attention_mask': torch.cat((torch.ones(input_ids.shape[1], dtype=torch.long), torch.zeros(512 - input_ids.shape[1], dtype=torch.long))).unsqueeze(0).squeeze()}
        return return_dict

In [153]:
torch.cat((torch.randn(1, 2), torch.randn(1,6)), 1)

tensor([[ 0.0243, -0.1559, -0.7643, -0.9884, -0.5549, -1.5908, -0.2357,  0.0341]])

In [169]:
a = process_dataset({"text": 'He said he inquired about the matter and was told that his wife got the notice because a Department of Motor Vehicles employee in Butler County coded the information incorrectly. He said the state should ensure the accuracy of its information before inviting millions of people to register.   “Why would you rush this mailing right before an election if it’s not recorded properly?” he asked. “What is going on is not OK.” '})  # "This was an [MASK]. This was not an aliquant. [REF-BEG] aliquot [REF-END]"})

In [170]:
a

{'input_ids': tensor([[  101,   103,   103,  2002, 24849,  2055,  1996,  3043,  1998,  2001,
           2409,  2008,  2010,  2564,  2288,  1996,   103,  2138,  1037,  2533,
            103,  5013,  4683,  7904,  1999,  7055,  2221, 22402,  1996,  2592,
          19721,  1012,  2002,   103,  1996,  2110,  2323,   103,   103, 10640,
           1997,  2049,   103,  2077,   103,  8817,   103,  2111,  2000,  4236,
           1012,  1523, 22677,  2052,  2017,  5481,  2023,  5653,  2075,   103,
           2077,  2019,  2602,  2065,  2009,  1521,   103,  2025,  2680,  7919,
           1029,  1524,  2002,  2356,   103,  1523,  2054,  2003,  2183,  2006,
           2003,  2025,  7929,  1012,  1524,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [172]:
len(a['input_ids'][0]), len(a['attention_mask'][0]), len(a['labels'][0])

(512, 512, 512)

In [130]:
path: str = "/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/processed/mixed_experiment/debug.txt"

In [136]:
data = datasets.load_dataset(
    "text", data_files=path, sample_by="line")

Found cached dataset text (/home/philko/.cache/huggingface/datasets/text/default-5125e412db2a48f1/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [173]:
# tokenize_function  = lambda examples: tokenizer(examples["text"])
ds = data.map(
    process_dataset,
    remove_columns=['text'])

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

In [177]:
for i in range(len(ds['train'])):
    print(len(ds['train'][i]['input_ids'][0]), len(ds['train'][i]['labels'][0]), len(ds['train'][i]['attention_mask'][0]))

512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 

In [52]:
a = tokenizer(data['train'][0]['text'], padding='max_length', max_length=512, truncation=True)

In [58]:
tokenizer.pad_token_type_id

0

In [57]:
a['input_ids']

[101,
 1996,
 8009,
 1997,
 8797,
 3544,
 2740,
 3771,
 2084,
 2453,
 2031,
 2042,
 11436,
 1024,
 3342,
 2129,
 9280,
 18704,
 2477,
 2790,
 2055,
 1996,
 8840,
 2497,
 2206,
 1996,
 3565,
 4605,
 2007,
 4311,
 2008,
 2957,
 11011,
 2052,
 2342,
 8999,
 5970,
 1010,
 2008,
 27829,
 7306,
 2453,
 2342,
 6181,
 5970,
 1998,
 4656,
 2726,
 2453,
 2342,
 3244,
 5970,
 1029,
 2612,
 1010,
 2069,
 2726,
 2018,
 5970,
 2007,
 7306,
 1998,
 11011,
 2025,
 2069,
 9992,
 1996,
 5442,
 2021,
 4760,
 2039,
 3929,
 7965,
 2005,
 27178,
 3022,
 1010,
 8019,
 1999,
 2673,
 1006,
 9131,
 2008,
 7306,
 2056,
 2002,
 1521,
 1055,
 2196,
 2371,
 2488,
 2012,
 2023,
 2391,
 1999,
 2019,
 2125,
 1011,
 2161,
 1007,
 1012,
 2726,
 1010,
 5564,
 1010,
 3544,
 2006,
 2650,
 2000,
 2022,
 3201,
 2005,
 1996,
 2927,
 1997,
 1996,
 2161,
 1010,
 2065,
 2025,
 1996,
 3098,
 1997,
 2731,
 3409,
 1010,
 2893,
 2041,
 2006,
 1996,
 2492,
 2005,
 1037,
 2261,
 2220,
 28308,
 1999,
 9857,
 1521,
 1055,
 3218,
 1012,


In [44]:
ds['train'][0:5]

{'input_ids': [[[101,
    1996,
    8009,
    1997,
    103,
    3544,
    2740,
    3771,
    2084,
    2453,
    2031,
    2042,
    11436,
    103,
    3342,
    2129,
    23479,
    103,
    103,
    2790,
    27588,
    1996,
    8840,
    2497,
    2206,
    1996,
    3565,
    4605,
    2007,
    4311,
    2008,
    103,
    11011,
    2052,
    2342,
    8999,
    5970,
    1010,
    2008,
    27829,
    7306,
    2453,
    2342,
    6181,
    5970,
    1998,
    103,
    2726,
    2453,
    2342,
    3244,
    5970,
    1029,
    2612,
    1010,
    103,
    2726,
    2018,
    5970,
    2007,
    7306,
    1998,
    11011,
    2025,
    2069,
    103,
    1996,
    5442,
    2021,
    4760,
    2039,
    3929,
    7965,
    2005,
    27178,
    3022,
    1010,
    8019,
    1999,
    2673,
    1006,
    9131,
    103,
    7306,
    2056,
    2002,
    1521,
    1055,
    2196,
    2371,
    2488,
    2012,
    2023,
    2391,
    1999,
    2019,
    103,
    103,
    2161,
  

In [47]:
tokenizer.pad(ds['train'][1], padding=True, return_tensors="pt", max_length=512)

{'input_ids': tensor([[  101,  2057,  2024,  2183,  2000,  2022,   103,   103,   103,  1012,
          2057,  2024,  2025,  2183,  2000,  2022,  2557,  7630, 13013,  1012,
           102]]), 'labels': tensor([[-100, -100, -100, -100, -100, -100, 2557, 4502, 4226, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [34]:
ds1 =  ds.map(tokenizer.pad, fn_kwargs={'return_tensors': 'pt'})

Loading cached processed dataset at /home/philko/.cache/huggingface/datasets/text/default-5125e412db2a48f1/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-9258d617351bdebf.arrow


In [39]:
len(ds1['train']["input_ids"][1][0])

21

In [19]:
for i in range(len(ds['train']['input_ids'])):
    print(len(ds['train']['input_ids'][i][0]))

199
32
79
183
24
178
30
86
28
57
79
137
24
26
149
156
27
155
227
28
28
35
151
162
23
222
97
30
28
23
168
153
156
187
226
76
118
27
170
68
310
165
25
194
28
28
125
178
204
80
29
25
85
67
30
106
30
114
32
233
27
108
28
52
28
104
71
26
31
27
63
111
197
108
141
188
88
75
131
46
27
27
142
25
29
32
27
104
162
28
129
28
118
69
170
76
22
32
176
122
91
26
34


In [13]:
ds = pd.read_csv(path, header=None, delimiter='NODELIMITERUSEDHEREJUSTREADLINEBYLINE')
ds.columns = ['text']
data: datasets.Dataset = datasets.Dataset.from_pandas(ds)


ds = data.map(
    process_dataset,
    new_fingerprint='asdasdasd')

/tmp/ipykernel_13581/3827118540.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ds = pd.read_csv(path, header=None, delimiter='NODELIMITERUSEDHEREJUSTREADLINEBYLINE')


Map:   0%|          | 0/1510400 [00:00<?, ? examples/s]

KeyError: 'input_ids'